<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Entrenamiento/Entrenamiento_YoloV8_Varios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image
import optuna
import locale
import os
import json
import pandas as pd
import yaml
import torch
import cv2
import random

# Verificación de CUDA
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de dispositivos CUDA disponibles:", torch.cuda.device_count())
    print("Nombre del dispositivo CUDA actual:", torch.cuda.get_device_name())

# Hiperparámetros ajustados para YOLO
HYPERPARAMETERS = {
    'epochs': 10000,
    'patience': 250,
    'batch': [8, 32],
    'workers': [16, 24],
    'imgsz': [320, 1024],
    'save_period': -1,
    'num_trials': 1,
    'lr0': [1e-5, 1e-2],
    'momentum': [0.7, 0.99],
    'weight_decay': [1e-5, 1e-2],
    'warmup_epochs': [100, 1000],
    'warmup_momentum': [0.0, 0.95],
    'warmup_bias_lr': [1e-5, 1e-2],
    'optimizer': ['RMSProp', 'Adam', 'SGD'],
    'testing_threshold': [0.4, 0.6]
}

# Combinaciones de datasets y pesos preentrenados
combinaciones = [
    {"ruta_resultados": "/content/modelos/YoloV8_m_5_fix", "archivo_yaml": "/content/drive/MyDrive/Colab_TFM/dataset/Polipos265_Detectron2FIX_YoloV8/data.yaml", "pre_weights": "yolov8m-seg.yaml"}
]

# Función objetivo para Optuna
def objective(trial, ruta_resultados, archivo_yaml, pre_weights):
    lr0 = trial.suggest_float('lr0', HYPERPARAMETERS['lr0'][0], HYPERPARAMETERS['lr0'][1], log=True)
    momentum = trial.suggest_float('momentum', HYPERPARAMETERS['momentum'][0], HYPERPARAMETERS['momentum'][1])
    weight_decay = trial.suggest_float('weight_decay', HYPERPARAMETERS['weight_decay'][0], HYPERPARAMETERS['weight_decay'][1], log=True)
    warmup_epochs = trial.suggest_int('warmup_epochs', HYPERPARAMETERS['warmup_epochs'][0], HYPERPARAMETERS['warmup_epochs'][1])
    warmup_momentum = trial.suggest_float('warmup_momentum', HYPERPARAMETERS['warmup_momentum'][0], HYPERPARAMETERS['warmup_momentum'][1])
    warmup_bias_lr = trial.suggest_float('warmup_bias_lr', HYPERPARAMETERS['warmup_bias_lr'][0], HYPERPARAMETERS['warmup_bias_lr'][1], log=True)
    optimizer = trial.suggest_categorical('optimizer', HYPERPARAMETERS['optimizer'])
    testing_threshold = trial.suggest_float('testing_threshold', HYPERPARAMETERS['testing_threshold'][0], HYPERPARAMETERS['testing_threshold'][1])
    batch_size = trial.suggest_int('batch', HYPERPARAMETERS['batch'][0], HYPERPARAMETERS['batch'][1], step=8)
    workers = trial.suggest_int('workers', HYPERPARAMETERS['workers'][0], HYPERPARAMETERS['workers'][1])
    imgsz = trial.suggest_int('imgsz', HYPERPARAMETERS['imgsz'][0], HYPERPARAMETERS['imgsz'][1], step=64)

    # Crear una instancia del modelo
    model = YOLO(pre_weights)

    # Configurar los parámetros de entrenamiento
    training_params = {
        'data': archivo_yaml,
        'project': ruta_resultados,
        'name': f"trial_{trial.number}",
        'epochs': HYPERPARAMETERS['epochs'],
        'patience': HYPERPARAMETERS['patience'],
        'batch': batch_size,
        'imgsz': imgsz,
        'lr0': lr0,
        'momentum': momentum,
        'weight_decay': weight_decay,
        'warmup_epochs': warmup_epochs,
        'warmup_momentum': warmup_momentum,
        'warmup_bias_lr': warmup_bias_lr,
        'optimizer': optimizer,
        'workers': workers,
        'save_period': HYPERPARAMETERS['save_period']
    }

    # Entrenar el modelo con los hiperparámetros sugeridos
    results = model.train(**training_params)

    # Definir la ruta para la carpeta del trial
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial.number}")
    if not os.path.exists(trial_dir):
        os.makedirs(trial_dir)

    # Definir la ruta para la carpeta del trial y el archivo all_metrics.json
    model_path = os.path.join(trial_dir, "weights/best.pt")

    # Cargamos el modelo que se acaba de entrenar
    model_segmentation = YOLO(model_path)

    # Evaluar el modelo en el conjunto de test con el umbral de confianza especificado
    val_params = {
        'data': archivo_yaml,
        'conf': testing_threshold,
        'save_json': True,
        'split': "test",
        'project': trial_dir,
        'name': "Evaluacion"
    }
    print("Antes de validar")
    test_results = model_segmentation.val(**val_params)
    print("Despues de validar")

    # Obtener las métricas del modelo
    metrics = test_results.results_dict

    # Crear un DataFrame con las métricas
    df = pd.DataFrame([metrics])

    # Extraer y convertir las métricas
    precision = df["metrics/precision(M)"].iloc[0]
    recall = df["metrics/recall(M)"].iloc[0]
    mAP50 = df["metrics/mAP50(M)"].iloc[0]

    # Imprimir las métricas para verificar
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"mAP: {mAP50}")

    # Crear un diccionario con las métricas relevantes
    metrics_row = {
        "Trial": trial.number,
        "Dataset": archivo_yaml,
        "Feedback": "mAP50",
        "Pre-Trained_Weights": pre_weights,
        "metrics": {
            'precision': metrics.get('metrics/precision(M)', 0),
            'recall': metrics.get('metrics/recall(M)', 0),
            'mAP50': metrics.get('metrics/mAP50(M)', 0),
            'mAP50-95': metrics.get('metrics/mAP50-95(M)', 0),
            'fitness': metrics.get('fitness', 0)
        },
        "hyperparameters": {
            'lr0': lr0,
            'epochs': HYPERPARAMETERS['epochs'],
            'batch_size': batch_size,
            'imgsz': imgsz,
            'momentum': momentum,
            'weight_decay': weight_decay,
            'warmup_epochs': warmup_epochs,
            'warmup_momentum': warmup_momentum,
            'warmup_bias_lr': warmup_bias_lr,
            'optimizer': optimizer,
            'workers': workers,
            'testing_threshold': testing_threshold
        }
    }
    metrics_path = os.path.join(trial_dir, "metrics.json")
    with open(metrics_path, 'w') as outfile:
        json.dump(metrics_row, outfile)

    return mAP50

# Bucle para ejecutar el entrenamiento y la optimización para cada combinación de rutas y pesos
for combinacion in combinaciones:
    ruta_resultados = combinacion["ruta_resultados"]
    archivo_yaml = combinacion["archivo_yaml"]
    pre_weights = combinacion["pre_weights"]

    # Ejecutar la optimización con Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, ruta_resultados, archivo_yaml, pre_weights), n_trials=HYPERPARAMETERS['num_trials'])

[I 2024-09-11 09:27:32,434] A new study created in memory with name: no-name-520c061f-9752-45fa-9d0a-a9975d87c740


CUDA disponible: True
Número de dispositivos CUDA disponibles: 1
Nombre del dispositivo CUDA actual: Tesla T4
Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.yaml, data=/content/drive/MyDrive/Colab_TFM/dataset/Polipos265_Detectron2FIX_YoloV8/data.yaml, epochs=5, time=None, patience=250, batch=16, imgsz=896, save=True, save_period=-1, cache=False, device=None, workers=9, project=/content/modelos/YoloV8_m_5_fix, name=trial_0, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

train: Scanning /content/drive/MyDrive/Colab_TFM/dataset/Polipos265_Detectron2FIX_YoloV8/train/labels.cache... 143 images, 0 backgrounds, 0 corrupt: 100%|██████████| 143/143 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Colab_TFM/dataset/Polipos265_Detectron2FIX_YoloV8/val/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]


...

val: Scanning /content/drive/MyDrive/Colab_TFM/dataset/Polipos265_Detectron2FIX_YoloV8/test/labels.cache... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.16s/it]

                   all         47         47          0          0          0          0          0          0          0          0


Speed: 0.4ms preprocess, 48.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/modelos/YoloV8_m_5_fix/trial_0/Evaluacion


[I 2024-09-11 09:29:45,394] Trial 0 finished with value: 0.0 and parameters: {'lr0': 7.920676778211194e-05, 'momentum': 0.7499602249493498, 'weight_decay': 0.0001760990697899691, 'warmup_epochs': 300, 'warmup_momentum': 0.6974233953137392, 'warmup_bias_lr': 0.006749351113156628, 'optimizer': 'SGD', 'testing_threshold': 0.5325605291443465, 'batch': 16, 'workers': 9, 'imgsz': 896}. Best is trial 0 with value: 0.0.


Despues de validar
Precision: 0.0
Recall: 0.0
mAP: 0.0


In [9]:
import shutil
# Directorio de origen
src_dir = "/modelos/YoloV8_m_5_fix"

# Directorio de destino
dest_dir = "/content/drive/MyDrive/Colab_TFM/modelos/YoloV8_m_5_fix"

def copy_directory(src, dest):
    if not os.path.exists(dest):
        os.makedirs(dest)

    for root, dirs, files in os.walk(src):
        for name in dirs:
            src_dir = os.path.join(root, name)
            dest_dir = os.path.join(dest, os.path.relpath(src_dir, src))
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)

        for name in files:
            src_file = os.path.join(root, name)
            dest_file = os.path.join(dest, os.path.relpath(src_file, src))
            if not os.path.exists(os.path.dirname(dest_file)):
                os.makedirs(os.path.dirname(dest_file))
            shutil.copy2(src_file, dest_file)

# Copiar el directorio de origen al directorio de destino
copy_directory(src_dir, dest_dir)

print(f"Directorio copiado de {src_dir} a {dest_dir}")

Directorio copiado de /modelos/YoloV8_m_5_fix a /content/drive/MyDrive/Colab_TFM/modelos/YoloV8_m_5_fix
